# Datos Climáticos web NOOA del Gobierno de EEUU

La descarga del data set se realiza en la URL http://www.ncdc.noaa.gov/orders/qclcd/. La información está contenida en ficheros comprimidos; cada uno de ellos alberga entre otros un fichero txt con la información de las precipitaciones.

Se pide tomar datos de varios años de este conjunto de datasets para las precipitaciones y obterner resúmenes:
- día en que ha habido más precipitaciones
- año en que ha habido más precipitaciones (obteniendo la media de cada año)

Se pide realizar el análisis en dos versiones:
- dataframes
- HDf5 

Procedimiento:
- importar librerías 
- descarga de los ficheros zip
- descompresión de los ficheros zip y selección de los txt relativos a precipitaciones: '.*precip*'
- construcción del dataframe
- realización de cálculos utilizando dataframes y HDF5   

In [1]:
from urllib import urlretrieve # copia de un network object denoted by a URL to en un fichero local

import zipfile # manipular archivos zip, no es válida si los files contienen comentarios

import os # métodos para trabajar de forma portable con las funcionalidades del sistema operativo
import os.path # acceso a ciertas funcionalidades relacionadas con los nombres de las rutas de archivos y directorios

import re # expresiones regulares

import glob # filename pattern matching

import numpy as np
import pandas as pd
from pandas import HDFStore, DataFrame #HDF5 con pandas

In [2]:
# Expresa el mes en un string que contenga dos caracteres para poder proceder a la descarga
# Así están nombrados los archivos en la URL

def string_month(mes):
    if len(str(mes)) == 1:
        return '0' + str(mes)
    else:
        return str(mes)

In [3]:
# Descomprime los ficheros de precipitaciones

def des_prep(zipFileName):
    
    # Compilación de la regular expression pattern into a regular expression object, para poder usar match()
    p = re.compile('.*precip*') 
    
    # Lee metadata de los archivos zip
    zfile = zipfile.ZipFile(zipFileName) 
    
    # Lee los nombres de los archivos en el zfile y devuelve una lista 
    for name in zfile.namelist():
        # Selecciona los files de precipitaciones
        if (p.match(name)): # Descomprimir los ficheros de precipitaciones
            #(dirname, filename) = os.path.split(name)
            zfile.extract(name, os.getcwd())

In [4]:
# Descarga los datos del año y llama a la función des_prep para la descompresión de los files de precipitaciones

def down_year(year):
    for i in range(1,13): # Este bucle solo funciona par aquellos años en los que haya zip para los doce meses
        urlretrieve('http://www.ncdc.noaa.gov/orders/qclcd/QCLCD'+str(year)+string_month(i)+'.zip',"QCLCD"+str(year)+string_month(i)+".zip")
        des_prep("QCLCD"+str(year)+string_month(i)+".zip")

## Descarga y creación de dataframe global
Vamos a probar con un año de crisis económica 2009 (año de los brotes verdes)  y otro de "recuperación económica", 2014. Es decir, "a lo loco". (Al probar más de dos años, algún archivo txt correspondiente a 2008 y 2015 tenía alguna columna adicional o algún comentario, no me ha sido posible realizar prueba y error, y entiendo que está fuera de lo exigido en el ejercicio). 

In [5]:
down_year(2009)

down_year(2014)


In [6]:
# Construye el dataframe
def concatenacion():   
    # Utilización de glob para la selección los txt del directorio de trabajo
    archivos = glob.glob(os.getcwd() + "/*precip*.txt") 
    frame = pd.DataFrame()
    lista = []
    for file_ in archivos:
        df = pd.read_csv(file_,index_col=None, header=0)
        lista.append(df)
    frame = pd.concat(lista) 
    return frame

In [7]:
df_prep = concatenacion()

In [8]:
df_prep.shape

(35675016, 5)

In [9]:
# Convertir a tipo numérico la variable Precipitation

df_prep['Precipitation_num'] = pd.to_numeric(df_prep['Precipitation'], errors = 'coerce')

In [10]:
df_prep.head(10)

,Wban,YearMonthDay,Hour,Precipitation,PrecipitationFlag,Precipitation_num
0,3011,20090101,1,,,NaN
1,3011,20090101,2,,,NaN
2,3011,20090101,3,,,NaN
3,3011,20090101,4,,,NaN
4,3011,20090101,5,,,NaN
5,3011,20090101,6,,,NaN
6,3011,20090101,7,,,NaN
7,3011,20090101,8,,,NaN
8,3011,20090101,9,,,NaN
9,3011,20090101,10,,,NaN


In [11]:
# Creacion de la variable Year

df_prep['Year'] = df_prep.YearMonthDay.apply(lambda x: str(x)[0:4])

In [12]:
df_prep.head(10)

,Wban,YearMonthDay,Hour,Precipitation,PrecipitationFlag,Precipitation_num,Year
0,3011,20090101,1,,,NaN,2009
1,3011,20090101,2,,,NaN,2009
2,3011,20090101,3,,,NaN,2009
3,3011,20090101,4,,,NaN,2009
4,3011,20090101,5,,,NaN,2009
5,3011,20090101,6,,,NaN,2009
6,3011,20090101,7,,,NaN,2009
7,3011,20090101,8,,,NaN,2009
8,3011,20090101,9,,,NaN,2009
9,3011,20090101,10,,,NaN,2009


In [13]:
# Limpieza del dataframe -variables no necesarias para el análisis-

df_prep_f = df_prep[['Precipitation_num','Year','YearMonthDay']]

In [14]:
df_prep_f.head(10)

,Precipitation_num,Year,YearMonthDay
0,NaN,2009,20090101
1,NaN,2009,20090101
2,NaN,2009,20090101
3,NaN,2009,20090101
4,NaN,2009,20090101
5,NaN,2009,20090101
6,NaN,2009,20090101
7,NaN,2009,20090101
8,NaN,2009,20090101
9,NaN,2009,20090101


In [15]:
# Limpieza del dataframe -NaN de la variable dependiente-

df_prep_f = df_prep_f[np.isfinite(df_prep_f['Precipitation_num'])]

In [16]:
df_prep_f.shape

(1615812, 3)

In [17]:
df_prep_f.head(10)

,Precipitation_num,Year,YearMonthDay
525,0.01,2009,20090122
531,0.01,2009,20090123
533,0.02,2009,20090123
534,0.02,2009,20090123
535,0.01,2009,20090123
541,0.01,2009,20090123
605,0.02,2009,20090126
606,0.02,2009,20090126
822,0.02,2009,20090104
823,0.03,2009,20090104


## Almacenamiento en formato HDF5

In [18]:
# Para la utilización de pandas y hdf5, he seguido: http://glowingpython.blogspot.com.es/2014/08/quick-hdf5-with-pandas.html
# Crear un fichero hdf5 y apertura en formato append 
hdf =HDFStore('hdf5_precipitacion')

# Colocar el dataframe df_prep_f en el fichero hdf5
hdf.put('Tabla/hdf5_precipitacion', df_prep_f, format='table', data_columns=True)

hdf['Tabla/hdf5_precipitacion'].head(10)

,Precipitation_num,Year,YearMonthDay
525,0.01,2009,20090122
531,0.01,2009,20090123
533,0.02,2009,20090123
534,0.02,2009,20090123
535,0.01,2009,20090123
541,0.01,2009,20090123
605,0.02,2009,20090126
606,0.02,2009,20090126
822,0.02,2009,20090104
823,0.03,2009,20090104


In [19]:
print hdf

<class 'pandas.io.pytables.HDFStore'>
File path: hdf5_precipitacion
/Tabla/hdf5_precipitacion            frame_table  (typ->appendable,nrows->1615812,ncols->3,indexers->[index],dc->[Precipitation_num,Year,YearMonthDay])


In [20]:
print hdf['Tabla/hdf5_precipitacion'].shape

(1615812, 3)


# Cálculo del día de máximas precipitaciones de los años seleccionados: 2009 y 2015

## Utilizando Dataframe

In [21]:
import datetime

# Para el cálculo del tiempo empleado
start = datetime.datetime.now()

# Día de máxima precipitación
df_group_by_day = df_prep_f[['YearMonthDay','Precipitation_num']].groupby('YearMonthDay').aggregate(np.sum)
print("Día"); print(df_group_by_day.idxmax())
print("Cantidad Precipitación"); print(df_group_by_day.max())

# Para el cálculo del tiempo empleado
stop = datetime.datetime.now()
Tiempo_DF = stop - start
print ("Tiempo empleado en el cálculo con DF"); print(Tiempo_DF)

Día
Precipitation_num    20141016
dtype: int64
Cantidad Precipitación
Precipitation_num    13010140.27
dtype: float64
Tiempo empleado en el cálculo con DF
0:00:00.104387


## Utilizando HDF5

In [22]:
start = datetime.datetime.now()

df_group_by_day = hdf['Tabla/hdf5_precipitacion'][['YearMonthDay','Precipitation_num']].groupby('YearMonthDay').aggregate(np.sum)
print("Día"); print(df_group_by_day.idxmax())
print("Cantidad Precipitación"); print(df_group_by_day.max())

# Para el cálculo del tiempo empleado
stop = datetime.datetime.now()
Tiempo_HDF = stop - start
print ("Tiempo empleado en el cálculo con HDP5"); print(Tiempo_HDF)


Día
Precipitation_num    20141016
dtype: int64
Cantidad Precipitación
Precipitation_num    13010140.27
dtype: float64
Tiempo empleado en el cálculo con HDP5
0:00:00.453963


# Cálculo de la media de precipitaciones por año: 2009 y 2015

## Utilizando Dataframe

In [23]:
start = datetime.datetime.now()

df_group_by_year = df_prep_f.groupby(['Year'])
df_group_by_year['Precipitation_num'].aggregate(np.mean)
Media_Anual  = df_group_by_year['Precipitation_num'].aggregate(np.mean)
print(Media_Anual)

stop = datetime.datetime.now()
Tiempo_DF = stop - start
print ("Tiempo empleado en el cálculo con DF"); print(Tiempo_DF)

Year
2009     0.066003
2014    30.783033
Name: Precipitation_num, dtype: float64
Tiempo empleado en el cálculo con DF
0:00:00.177153


## Utilizando HDF5

In [24]:
start = datetime.datetime.now()

df_group_by_year = hdf['Tabla/hdf5_precipitacion'].groupby(['Year'])
df_group_by_year['Precipitation_num'].aggregate(np.mean)
print(df_group_by_year['Precipitation_num'].aggregate(np.mean))

stop = datetime.datetime.now()
Tiempo_HDF = stop - start
print ("Tiempo empleado en el cálculo con HDF5"); print(Tiempo_HDF)

Year
2009     0.066003
2014    30.783033
Name: Precipitation_num, dtype: float64
Tiempo empleado en el cálculo con HDF5
0:00:00.386121


# Cálculo del tamaño en disco

In [25]:
print("Tamaño en memoria"); print(df_prep_f.size)
print("Tamaño en disco"); print(hdf.root.Tabla.hdf5_precipitacion.table.size_on_disk)

Tamaño en memoria
4847436
Tamaño en disco
45349528


## Opcional: guardar el resumen en una tabla HDF5 y ver el tiempo que se tarda en recuperarlo

In [26]:
# He utilizado la información contendida en http://pybonacci.org/2013/07/04/aprende-funcionalidad-basica-de-pytables-paso-a-paso-i/
# Pytables está programado sobre el formato hdf5 con ayuda de Python y Numpy

import tables as tb

# Creación de un fichero hdf5 en modo append
h5file = tb.open_file("hdf5_size", mode="a", Title="Por fin estoy acabando la práctica")

from tables import IsDescription,  Int64Col, Float64Col 

dt = np.dtype([('Year', int),('Prep', float)])
rows = np.array([(2009, Media_Anual[0]), (2014, Media_Anual[1])], dtype=dt)

# Creación de una tabla con la estructura creada con el constructor dt
h5file.create_table("/", "prep", dt)

# Append row
h5file.root.prep.append(rows)


start = datetime.datetime.now()
print ("Media Anual 2009")
print h5file.root.prep[0]
print ("Media Anual 2014")
print h5file.root.prep[1]

Media Anual 2009
(2009, 0.0660034185048455)
Media Anual 2014
(2014, 30.783032757442573)


In [27]:
for node in h5file:
    print(node)

/ (RootGroup) ''
/prep (Table(2,)) ''


In [28]:
# Cierre del HDF5

h5file.close

<bound method File.close of File(filename=hdf5_size, title='', mode='a', root_uep='/', filters=Filters(complevel=0, shuffle=False, fletcher32=False, least_significant_digit=None))
/ (RootGroup) ''
/prep (Table(2,)) ''
  description := {
  "Year": Int64Col(shape=(), dflt=0, pos=0),
  "Prep": Float64Col(shape=(), dflt=0.0, pos=1)}
  byteorder := 'little'
  chunkshape := (4096,)
>